# Código de Procesamiento de Imágenes

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Directorio de las imágenes
image_dir = "3d57_agg"

# Lista de archivos de imagen
image_files = [file for file in os.listdir(image_dir) if file.endswith('.png')]

def process_image(image_path):
    # Cargar la imagen en escala de grises
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"No se pudo cargar la imagen: {image_path}")
        return None
    
    # Filtrar valores representativos
    mask = image > 127
    
    # Obtener las coordenadas de los píxeles que cumplen la condición
    coords = np.column_stack(np.where(mask))
    
    if coords.size == 0:
        print(f"No se encontraron valores representativos en la imagen: {image_path}")
        return None

    # Determinar los límites de la caja delimitadora
    min_y, min_x = np.min(coords, axis=0)
    max_y, max_x = np.max(coords, axis=0)
    
    return (min_x, min_y), (max_x, max_y)

def plot_bounding_box(image_path, bbox):
    image = cv2.imread(image_path)
    if image is None:
        print(f"No se pudo cargar la imagen: {image_path}")
        return
    
    (min_x, min_y), (max_x, max_y) = bbox
    
    # Dibujar la caja delimitadora en la imagen
    cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
    
    # Mostrar la imagen con la caja delimitadora
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    bbox = process_image(image_path)
    if bbox:
        print(f"Caja delimitadora para {image_file}: {bbox}")
        plot_bounding_box(image_path, bbox)

# Código para Inspección de Imágenes

In [ ]:
def inspect_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"No se pudo cargar la imagen: {image_path}")
        return
    
    plt.figure(figsize=(12, 6))
    
    # Mostrar imagen
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title(f"Imagen: {os.path.basename(image_path)}")
    
    # Mostrar histograma
    plt.subplot(1, 2, 2)
    plt.hist(image.ravel(), bins=256, range=(0, 256))
    plt.title("Histograma de Intensidad")
    
    plt.show()

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    inspect_image(image_path)


```
Las imágenes y sus histogramas de intensidad nos muestran la distribución de los valores de píxeles en cada imagen. Observando los histogramas, podemos ajustar nuestro umbral de filtrado para detectar valores representativos más adecuados.
```

##  Usuario 3d57_agg

In [3]:
import cv2
import numpy as np
import os
from datetime import datetime
import paho.mqtt.client as mqtt

def enhance_contrast(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

def find_bounding_box(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"No se pudo cargar la imagen: {image_path}")
        return (0, 0, 0, 0), 0.0
    
    image = enhance_contrast(image)
    
    threshold = 100
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    points = np.column_stack(np.where(binary_image > threshold))
    
    if points.size == 0:
        print(f"No se encontraron píxeles representativos en la imagen: {image_path}")
        return (0, 0, 0, 0), 0.0
    
    min_y, min_x = points.min(axis=0)
    max_y, max_x = points.max(axis=0)
    
    uncertain = float(image.max()) / 255.0
    
    return (min_x, min_y, max_x, max_y), uncertain

def process_user_images(user_id, image_dir):
    bounding_boxes = []
    for image_file in os.listdir(image_dir):
        if image_file.endswith(".png"):
            image_path = os.path.join(image_dir, image_file)
            bbox, uncertain = find_bounding_box(image_path)
            timestamp = os.path.splitext(image_file)[0].split('.')[-1]
            timestamp_iso = datetime.utcfromtimestamp(int(timestamp)).isoformat()
            bounding_boxes.append({
                "timestamp_sensor": timestamp_iso,
                "x1": bbox[0],
                "x2": bbox[2],
                "y1": bbox[1],
                "y2": bbox[3],
                "uncertain": str(uncertain),
                "user_id": user_id
            })
    return bounding_boxes

user_directories = {
    "3d57_agg": "3d57_agg"
}

all_bounding_boxes = []
for user_id, image_dir in user_directories.items():
    if os.path.exists(image_dir):
        bounding_boxes = process_user_images(user_id, image_dir)
        all_bounding_boxes.extend(bounding_boxes)

mqtt_broker_host = "84.122.5.56"
mqtt_broker_port = 29002
location_id_base = "locationUser"
location_id_counter = 1

client = mqtt.Client()

client.connect(mqtt_broker_host, mqtt_broker_port, keepalive=60)
client.loop_start()

try:
    for bbox in all_bounding_boxes:
        message = f"timestamp_sensor|{bbox['timestamp_sensor']}|x1|{bbox['x1']}|x2|{bbox['x2']}|y1|{bbox['y1']}|y2|{bbox['y2']}|uncertain|{bbox['uncertain']}|user_id|{bbox['user_id']}"
        
        location_id = f"{location_id_base}{bbox['user_id']}{location_id_counter:03d}"
        topic = f"/ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/{location_id}/attrs"

        client.publish(topic, message, qos=2)
        
        location_id_counter += 1

except KeyboardInterrupt:
    print("Programa detenido por el usuario.")
    client.disconnect()


No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383260.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383500.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383200.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383620.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715382660.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715382780.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383020.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383680.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715382900.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383380.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383740.png
No se encontraron píxeles representativos en la imagen: 3d57_agg/FP.1715383140.png
No s

##  Usuario ed9c_agg

In [2]:
import cv2
import numpy as np
import os
from datetime import datetime
import paho.mqtt.client as mqtt

def enhance_contrast(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

def find_bounding_box(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"No se pudo cargar la imagen: {image_path}")
        return (0, 0, 0, 0), 0.0
    
    image = enhance_contrast(image)
    
    threshold = 100
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    points = np.column_stack(np.where(binary_image > threshold))
    
    if points.size == 0:
        print(f"No se encontraron píxeles representativos en la imagen: {image_path}")
        return (0, 0, 0, 0), 0.0
    
    min_y, min_x = points.min(axis=0)
    max_y, max_x = points.max(axis=0)
    
    uncertain = float(image.max()) / 255.0
    
    return (min_x, min_y, max_x, max_y), uncertain

def process_user_images(user_id, image_dir):
    bounding_boxes = []
    for image_file in os.listdir(image_dir):
        if image_file.endswith(".png"):
            image_path = os.path.join(image_dir, image_file)
            bbox, uncertain = find_bounding_box(image_path)
            timestamp = os.path.splitext(image_file)[0].split('.')[-1]
            timestamp_iso = datetime.utcfromtimestamp(int(timestamp)).isoformat()
            bounding_boxes.append({
                "timestamp_sensor": timestamp_iso,
                "x1": bbox[0],
                "x2": bbox[2],
                "y1": bbox[1],
                "y2": bbox[3],
                "uncertain": str(uncertain),
                "user_id": user_id
            })
    return bounding_boxes

user_directories = {
    "ed9c_agg": "ed9c_agg"
}

all_bounding_boxes = []
for user_id, image_dir in user_directories.items():
    if os.path.exists(image_dir):
        bounding_boxes = process_user_images(user_id, image_dir)
        all_bounding_boxes.extend(bounding_boxes)

mqtt_broker_host = "localhost"
mqtt_broker_port = 1883
location_id_base = "locationUser"
location_id_counter = 1

client = mqtt.Client()

client.connect(mqtt_broker_host, mqtt_broker_port, keepalive=60)
client.loop_start()

try:
    for bbox in all_bounding_boxes:
        message = f"timestamp_sensor|{bbox['timestamp_sensor']}|x1|{bbox['x1']}|x2|{bbox['x2']}|y1|{bbox['y1']}|y2|{bbox['y2']}|uncertain|{bbox['uncertain']}|user_id|{bbox['user_id']}"
        
        location_id = f"{location_id_base}{bbox['user_id']}{location_id_counter:03d}"
        topic = f"/ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/{location_id}/attrs"

        client.publish(topic, message, qos=2)
        
        location_id_counter += 1

except KeyboardInterrupt:
    print("Programa detenido por el usuario.")
    client.disconnect()


No se encontraron píxeles representativos en la imagen: ed9c_agg/FP.1715364840.png
No se encontraron píxeles representativos en la imagen: ed9c_agg/FP.1715363520.png
No se encontraron píxeles representativos en la imagen: ed9c_agg/FP.1715367420.png


##  Usuario f02e_agg

In [3]:
import cv2
import numpy as np
import os
from datetime import datetime
import paho.mqtt.client as mqtt

def enhance_contrast(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

def find_bounding_box(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"No se pudo cargar la imagen: {image_path}")
        return (0, 0, 0, 0), 0.0
    
    image = enhance_contrast(image)
    
    threshold = 100
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    points = np.column_stack(np.where(binary_image > threshold))
    
    if points.size == 0:
        print(f"No se encontraron píxeles representativos en la imagen: {image_path}")
        return (0, 0, 0, 0), 0.0
    
    min_y, min_x = points.min(axis=0)
    max_y, max_x = points.max(axis=0)
    
    uncertain = float(image.max()) / 255.0
    
    return (min_x, min_y, max_x, max_y), uncertain

def process_user_images(user_id, image_dir):
    bounding_boxes = []
    for image_file in os.listdir(image_dir):
        if image_file.endswith(".png"):
            image_path = os.path.join(image_dir, image_file)
            bbox, uncertain = find_bounding_box(image_path)
            timestamp = os.path.splitext(image_file)[0].split('.')[-1]
            timestamp_iso = datetime.utcfromtimestamp(int(timestamp)).isoformat()
            bounding_boxes.append({
                "timestamp_sensor": timestamp_iso,
                "x1": bbox[0],
                "x2": bbox[2],
                "y1": bbox[1],
                "y2": bbox[3],
                "uncertain": str(uncertain),
                "user_id": user_id
            })
    return bounding_boxes

user_directories = {
    "f02e_agg": "f02e_agg"
}

all_bounding_boxes = []
for user_id, image_dir in user_directories.items():
    if os.path.exists(image_dir):
        bounding_boxes = process_user_images(user_id, image_dir)
        all_bounding_boxes.extend(bounding_boxes)

mqtt_broker_host = "localhost"
mqtt_broker_port = 1883
location_id_base = "locationUser"
location_id_counter = 1

client = mqtt.Client()

client.connect(mqtt_broker_host, mqtt_broker_port, keepalive=60)
client.loop_start()

try:
    for bbox in all_bounding_boxes:
        message = f"timestamp_sensor|{bbox['timestamp_sensor']}|x1|{bbox['x1']}|x2|{bbox['x2']}|y1|{bbox['y1']}|y2|{bbox['y2']}|uncertain|{bbox['uncertain']}|user_id|{bbox['user_id']}"
        
        location_id = f"{location_id_base}{bbox['user_id']}{location_id_counter:03d}"
        topic = f"/ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/{location_id}/attrs"

        client.publish(topic, message, qos=2)
        
        location_id_counter += 1

except KeyboardInterrupt:
    print("Programa detenido por el usuario.")
    client.disconnect()


No se encontraron píxeles representativos en la imagen: f02e_agg/FP.1715367300.png
No se encontraron píxeles representativos en la imagen: f02e_agg/FP.1715367180.png
No se encontraron píxeles representativos en la imagen: f02e_agg/FP.1715385480.png
No se encontraron píxeles representativos en la imagen: f02e_agg/FP.1715367240.png


In [ ]:
%reset -f

In [ ]:
all_bounding_boxes